# Discriminative Biclustering Algorithm 
Proposed by Odibat & Reddy, 2014 in **Efficient mining of discriminative co-clusters from gene
expression data**

In [1]:
%load_ext pycodestyle_magic

In [255]:
%matplotlib inline
import numpy as np
import math
from matplotlib import pyplot as plt
from sklearn.metrics import consensus_score
import multiprocessing as mp
import pandas as pd
import glob as glob
from functools import reduce

In [4]:
# !pip install pycodestyle
# !pip install pycodestyle_magic

### Definition 1 - Coherence Measure H

In [3]:
#%pycodestyle
class CoherenceMeasure(object):
    def __init__(self, data):
        self.data = data
        self.n, self.m = data.shape
        self.xiJ = np.mean(data, axis=1)
        self.xIj = np.mean(data, axis=0)
        self.xIJ = np.mean(data)
        self._H = None

    @property
    def H(self):
        if self._H is None:
            print("Computing coherence measure")
            self._H = self._compute_H()
            print("H value: " + str(self._H))
        return self._H
    
    def _compute_H(self):
        H = 0
        for i in range(self.n):
            for j in range(self.m):
                H += (self.data[i, j] - self.xIj[j] -
                      self.xiJ[i] + self.xIJ)**2
        H *= 1.0/math.fabs(self.m*self.n)
        H = 1 - H
        return H

#### Loading test data for Coherence Measure

In [4]:
import random
data = np.random.random((50, 50))
print(data)

[[ 0.67223392  0.05730369  0.58718702 ...,  0.46175495  0.35493749
   0.12153704]
 [ 0.80887785  0.0451742   0.34966532 ...,  0.40622315  0.66183642
   0.86556317]
 [ 0.67203729  0.12295668  0.59680418 ...,  0.018873    0.00771753
   0.02325043]
 ..., 
 [ 0.56439024  0.58408646  0.43073384 ...,  0.93899258  0.37981884
   0.46816191]
 [ 0.58067125  0.9237371   0.55683491 ...,  0.34105763  0.52530834
   0.23168843]
 [ 0.92655025  0.61445531  0.30605175 ...,  0.32990357  0.27647049
   0.82275964]]


In [5]:
# Testing Coherence
coherence_measure = CoherenceMeasure(data)
print("H = " + str(coherence_measure.H))

Computing coherence measure
H value: 0.919075202837
H = 0.919075202837


### Definition 2 - Positive and negative correlations

In [6]:
# input: rows x and y and J columns
# output: positive and negative correlations


class PositiveNegativeCorrelation(object):
    def __init__(self, x, y, J):
        self._x = x
        self._y = y
        self._J = J
        self._x_mean = np.mean(x)
        self._y_mean = np.mean(y)
        self._H_pos = None
        self._H_neg = None

    @property
    def H_pos(self):
        if self._H_pos is None:
            # print("Computing H positive...")
            self._H_pos = self._compute_H_pos()
            # print("H positive value: " + str(self._H_pos))
        return self._H_pos

    @property
    def H_neg(self):
        if self._H_neg is None:
            # print("Computing H negative...")
            self._H_neg = self._compute_H_neg()
            # print("H negative value: " + str(self._H_neg))
        return self._H_neg

    def _compute_H_pos(self):
        H_pos = 0
        for j in range(self._J):
            aux = (((self._x[j] - self._x_mean) -
                    (self._y[j] - self._y_mean))/2.0)**2
            H_pos += aux
        H_pos *= 1.0/math.fabs(self._J)
        H_pos = 1 - H_pos
        return H_pos

    def _compute_H_neg(self):
        H_neg = 0
        for j in range(self._J):
            aux = (((self._x[j] - self._x_mean) +
                    (self._y[j] - self._y_mean))/2.0)**2
            H_neg += aux
        H_neg *= 1.0/math.fabs(self._J)
        H_neg = 1 - H_neg
        return H_neg

#### Loading test data for positive and negative correlation

In [7]:
x = np.random.random((5))
y = np.random.random((5))
J = 5
print("Row x " + str(x))
print("Row y " + str(y))
print("J value " + str(J))

Row x [ 0.93895089  0.11710639  0.32643905  0.4463099   0.27790122]
Row y [ 0.89265351  0.64024148  0.12252775  0.09832966  0.67569515]
J value 5


In [8]:
# Testing correlation
positive_negative_correlation = PositiveNegativeCorrelation(x,y,J)
print("H positive " + str(positive_negative_correlation.H_pos))
print()
print("H negative " + str(positive_negative_correlation.H_neg))

H positive 0.97120541997

H negative 0.938972024938


### Definition 3 - Pair-based coherence

In [119]:
#%%pycodestyle

# input: co-cluster X of I rows and J columns
# output: paired-based coherence


class PairBasedCoherence(object):
    def __init__(self, X):
        self._X = np.array(X)
        self._I, self._J = X.shape
        self._HP = None

    @property
    def HP(self):
        if self._HP is None:
            # print("Calculating Pair based coherence..")
            self._HP = self._compute_HP_()
            # print("Paired based coherence value: " + str(self._HP))
        return self._HP

    def _compute_HP_(self):
        HP = 0
        for i in range(self._I):
            for j in range(i+1, self._I):
                if (i==j): 
                    break
                x = self._X[i]
                y = self._X[j]
                correlation = PositiveNegativeCorrelation(x, y,self._J)
                H0 = correlation.H_pos
                # H0 = max(correlation.H_pos,correlation.H_neg)
                HP += H0
        HP *= math.fabs(1.0)/(math.fabs(self._I)*(math.fabs(self._I)-1))
        return HP

#### Loading test data for pair-based coherence

In [10]:
data = np.random.random((50, 50))
print(data)

[[ 0.3977003   0.41097539  0.92490071 ...,  0.42671076  0.15869944
   0.23010634]
 [ 0.49486328  0.97289265  0.16373207 ...,  0.89676011  0.04975584
   0.15219219]
 [ 0.07953859  0.39669464  0.60855359 ...,  0.22288821  0.38088693
   0.24439542]
 ..., 
 [ 0.74989084  0.61744422  0.4867372  ...,  0.24092302  0.87401566
   0.86000712]
 [ 0.92654578  0.16637444  0.57521542 ...,  0.63668307  0.58885201
   0.12506129]
 [ 0.07881462  0.87313589  0.57826627 ...,  0.04329009  0.76787694
   0.09478329]]


In [11]:
pair_based_coherence = PairBasedCoherence(data)
print("H value " + str(pair_based_coherence.HP))

Paired based coherence value: 0.479712387334
H value 0.479712387334


### Coherence for a new z in in X

In [12]:
%%latex
Define H for a new term 'z' in X
$$
H_{1}(I,J,X,z) =H_{0}(I,J,X) \cdot \frac{(I-1)}{(I+1)} + \frac{|2|}{(I)(I+1)} \cdot \sum_{x \epsilon X} {h(x,z,J)}
$$

<IPython.core.display.Latex object>

### RAPOOC

This algorithm is proposed to efficiently extract the most coherent and large co-clusters that area arbitrarily positioned in the data matrix.



#### Algorithm 1 RAPOOC (D,k,l,K)
Input: Data matrix D, number of row clusters (k), number of column clusters (l), number of optimized co-clusters (K)

Output: A set of K co-clusters({X})

In [13]:
data = pd.read_csv('TestData/SimulatedDataCoherence/HighCoherenceMix.csv',header=None)
data.head()

,0,1,2,3,4,5,6
0,0.015521,0.013931,0.49935,0.98560,0.49268,0.006029,0.010850
1,0.007642,0.007915,0.49475,0.98089,0.49296,0.008737,0.007426
2,0.009314,0.000106,0.50717,0.98620,0.49783,0.007664,0.016632
3,0.003243,0.006791,0.50297,0.98170,0.50450,0.004663,0.018980
4,0.010901,0.011237,0.49306,0.98213,0.50162,0.017864,0.001509


In [102]:

class BisectingClusterer(object):
    def __init__(self, data):
        if data is not None:
            self._data = np.array(data)
            self._I, self._J = self._data.shape
        else:
            print("Empty data")
    
    @property
    def centroids(self):
        return self._centroids

    def fit(self):
        self._centroids = self._compute_centroids_()
        bisecting_indices = self._bisect_clusters_(self._centroids)
        return bisecting_indices
    
    def _compute_centroids_(self):
        max_correlation = 0
        centroids = [0,0]
        for i in range(self._I):
            for j in range(i+1, self._I):
                if (i == j):
                    break
                correlation = PositiveNegativeCorrelation(self._data[i],
                                                          self._data[j],
                                                          self._J).H_neg
                if(correlation > max_correlation):
                    max_correlation = correlation
                    centroids[0] = i
                    centroids[1] = j
        return centroids

    def _bisect_clusters_(self, centroids):
        cluster_indices = np.zeros(self._I)
        for i in range(self._I):
            correlation0 = PositiveNegativeCorrelation(
                self._data[centroids[0]], self._data[i],self._J).H_pos
            correlation1 = PositiveNegativeCorrelation(
                self._data[centroids[1]], self._data[i],self._J).H_pos
            if(correlation0 <= correlation1):
                cluster_indices[i] = 1
        return cluster_indices

In [267]:
class Rapooc(object):
    def __init__(self, D, k, l, K):
        self._D = np.array(D)
        assert k>0 and l >0, "invalid values, k>0 and l>0"
        self._k = k
        self._l = l
        assert K <= k*l and k>=1, "invalid values, 1<= K <= k*l"
        self._K = K
        self._rho = np.ones(D.shape[0])
        self._gamma = np.ones(D.shape[1])
        self._M, self._N = self._D.shape
        self._cluster_H_values = None
            
    @property
    def rho(self):
        return self._rho
    
    @property
    def gamma(self):
        return self._gamma
    
    @property
    def bicluster_h_values(self):
        return self._cluster_H_values

    def initialize(self):
        i = 1
        j = 1
        while (i < self._k or j < self._l):
            if i < self._k:
                i += 1
                alpha = self._argmin_H_(self._rho, self._gamma,'row')
                self._bisect_partitions_(self._D[np.where(self._rho==alpha)], self._rho, alpha, i,'row')
            if j < self._l:
                j += 1
                beta = self._argmin_H_(self._rho,self._gamma, 'column')
                self._bisect_partitions_((self._D.T)[np.where(self._gamma==beta)], self._gamma, beta, j,'column')   

    def _argmin_H_(self, row_co_cluster, col_co_cluster,option='row'):
        if (option=='row'):
            data = self._D
        else:
            data = self._D.T
        h_min = math.inf
        min_cocluster = 1
        map_array = np.int64(row_co_cluster if option == 'row' else col_co_cluster)
        max_index_in_map = np.max(map_array)
        for i in range(1,max_index_in_map):
            if (list(map_array).count(i) < 2):
                pass
            else:
                coherence = PairBasedCoherence(
                    data[np.where((row_co_cluster if option == 'row' else col_co_cluster) == i)]).HP 
                if (coherence <= h_min):
                    h_min = coherence
                    min_cocluster = i
        return min_cocluster

    def _bisect_partitions_ (self, data, mapping_array, cluster_to_replace, new_cluster_index,option='row'):
        clusterer = BisectingClusterer(data)
        bisected_map = clusterer.fit()
        bisected_map_index = 0
        for i in range(0,len(mapping_array)):
            if ((self._rho if option == 'row' else self._gamma)[i] == cluster_to_replace):
                if (bisected_map[bisected_map_index] == 1.0):
                    (self._rho if option == 'row' else self._gamma)[i] = new_cluster_index
                bisected_map_index += 1
                          
    def core_coclustering(self):
        # self.compute_H_values()
        max_h_value = 0.0
        h_values_repetitions = 0
        flg_h_drop = False
        
        #simulated annealing?
        rows_delta = 0.0
        cols_delta = 0.0
        for i in range(0,10):        
            
            if (h_values_repetitions>20 or flg_h_drop):
                break;
                
            # jobs
            jobs = list()
            jobs.append(mp.Process(target = self._arg_max_, args = ('row',rows_delta)))
            jobs.append(mp.Process(target = self._arg_max_, args = ('column',cols_delta))) 
            
            for job in jobs:
                job.start()
            
            for job in jobs:
                job.join()
            
            #join jobs
            actual_h_value = rows_delta + cols_delta
            
            if(actual_h_value > max_h_value):
                max_h_value = actual_h_value
                h_values_repetitions = 0
            elif (math.fabs(actual_h_value - max_h_value) <= 0.001):
                h_values_repetitions += 1
            else:
                h_values_repetitions = 0
                
    def compute_H_values(self):
        self._cluster_H_values = np.array([ [0.0] * int(max(self._rho) + 1)  
                                           for _ in range(int(max(self._gamma) + 1))])
        for i in range(1,int(max(self._rho) + 1)):
            for j in range (1, int(max(self._gamma) + 1)):
                coherence = PairBasedCoherence(self._D[np.ix_(np.where(self._rho == i)[0],
                                                              np.where(self._gamma == j)[0])]).HP
                self._cluster_H_values[i][j] = coherence
            
    def _row_clustering_(self):
            self._rho = self._arg_max_('row')
    
    def _column_clustering_(self):
            self._gamma = self._arg_max_('column')
            
    def _arg_max_(self,option = 'row',return_value = 0): 
        
        if option == 'row':
            mapping_array = self._rho
            data = self._D
        else:
            mapping_array = self._gamma
            data = self._D.T
            
        num_of_biclusters = int(np.max(mapping_array))
        
        # initial_optimum = reduce(lambda x,y: x + y, h_values[1:int(max(mapping_array))])
        
        total_delta = 0
        for i in range (0,len(data)):
            flg_keep = True
            element = data[i]
            element_cluster = int(mapping_array[i])
            element_h = self._compute_single_h_term_(element_cluster, data, mapping_array)
            optimal_delta = 0
            if (list(mapping_array).count(element_cluster)>2):
                no_element_h = self._compute_single_h_term_(element_cluster, data, mapping_array)
                mapping_array[i] = 0
                flg_keep = (math.fabs(element_h - no_element_h) <= 0.001 )
            else: 
                flg_keep = True
            optimal_cluster = 0
            flg_h_rise = False
            for cluster in range(1,int(max(mapping_array))):
                if (cluster == element_cluster):
                    pass 
                else:
                    if ((list(mapping_array).count(i)) > 2):
                        mapping_array[i] = cluster
                        cluster_size = list(mapping_array).count(cluster)
                        h_delta = self._compute_single_h_term_(cluster, data, mapping_array)
                        if (h_delta > optimal_delta):
                            optimal_delta = h_delta
                            optimal_cluster = cluster
                            flg_h_rise = True

            mapping_array[i] =  optimal_cluster if \
                            flg_h_rise else \
                            (element_cluster if flg_keep else 0)
            total_delta += optimal_delta
        return_value = optimal_delta
        
    def _compute_single_h_term_(self, cluster, data, mapping_array):
        coherence = PairBasedCoherence(data[np.where(mapping_array == cluster)]).HP
        return coherence

In [268]:
rapooc = Rapooc(data,2,2,4)
rapooc.initialize()

In [269]:
rapooc.core_coclustering()

Process Process-14:
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/a20125359/anaconda3/envs/Biclustering/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/a20125359/anaconda3/envs/Biclustering/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-267-23eec3b67539>", line 141, in _arg_max_
    no_element_h = self._compute_single_h_term_(element_cluster, data, mapping_array)


KeyboardInterrupt: 

  File "<ipython-input-267-23eec3b67539>", line 166, in _compute_single_h_term_
    coherence = PairBasedCoherence(data[np.where(mapping_array == cluster)]).HP
  File "<ipython-input-119-a59eefaa546f>", line 17, in HP
    self._HP = self._compute_HP_()
  File "<ipython-input-119-a59eefaa546f>", line 29, in _compute_HP_
    correlation = PositiveNegativeCorrelation(x, y,self._J)
  File "<ipython-input-6-1b51c1b25359>", line 11, in __init__
    self._y_mean = np.mean(y)
  File "/home/a20125359/anaconda3/envs/Biclustering/lib/python3.6/site-packages/numpy/core/fromnumeric.py", line 2909, in mean
    out=out, **kwargs)
  File "/home/a20125359/anaconda3/envs/Biclustering/lib/python3.6/site-packages/numpy/core/_methods.py", line 66, in _mean
    elif issubclass(arr.dtype.type, nt.float16):


In [ ]:
rapooc.bicluster_h_values

In [246]:
rapooc.gamma

array([ 1.,  1.,  1.,  2.,  2.,  1.,  1.])

In [247]:
rapooc.rho

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2

In [ ]:
import glob
glob.glob('TestData/SimulatedDataCoherence/*')

In [76]:
d = np.array(np.array([0,1,2,3,3,3,6,7,8]))
a = d[np.whe]

In [139]:
np.array(data)[np.ix_([0,3],[1,2])]

array([[ 0.013931 ,  0.49935  ],
       [ 0.0067908,  0.50297  ]])

In [72]:
for i in range(1,2):
    print(i)

1
